# Stats Project - Valuations Datasets

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from datetime import datetime 

In [2]:
pd.set_option('display.max_rows', 500)

## Load all data

In [3]:
# Appearances 
appearances = pd.read_csv('transfermarkt_data/appearances.csv')

# Games 
club_games = pd.read_csv('transfermarkt_data/club_games.csv')
clubs = pd.read_csv('transfermarkt_data/clubs.csv')

# Competition
competition = pd.read_csv('transfermarkt_data/competitions.csv')

# Games 
game_events = pd.read_csv('transfermarkt_data/game_events.csv')
games = pd.read_csv('transfermarkt_data/games.csv')

# Players 
player_valuations = pd.read_csv('transfermarkt_data/player_valuations.csv')
players = pd.read_csv('transfermarkt_data/players.csv')

# Transfers 
transfers = pd.read_csv('transfermarkt_data/transfers.csv')

### Subsetting appearnaces, and creating a player-club-season dataframe

In [4]:
appearnaces_subset = appearances[appearances['competition_id'].isin(['ES1', 'IT1', 'L1', 'FR1', 'GB1'])]
appearnaces_subset[['player_id', 'player_name', 'player_club_id', 'date', 'competition_id']].sort_values(by=['player_id', 'date'])

# Step 1: Convert 'date' to datetime
appearnaces_subset['date'] = pd.to_datetime(appearnaces_subset['date'])

# Step 2: Extract the season# Step 2: Define football season logic
def get_football_season(date):
    year = date.year
    if date.month >= 7:  # If July or later, season starts in current year
        return f"{year}-{year + 1}"
    else:  # If before July, season belongs to previous year
        return f"{year - 1}-{year}"

# Apply the function to create the 'season' column
appearnaces_subset['season'] = appearnaces_subset['date'].apply(get_football_season)

# Step 3: Map competition codes to leagues
league_mapping = {'IT1': 'SeriaA', 'ES1': 'LaLiga', 'L1': 'Bundesliga', 'FR1': 'Ligue1', 'GB1': 'EPL'}
appearnaces_subset['league'] = appearnaces_subset['competition_id'].map(league_mapping)

# Step 4: Rename columns for clarity
appearnaces_subset = appearnaces_subset.rename(columns={'player_club_id': 'team', 'player_name': 'player'})

/var/folders/xk/dkwzfs8166d33myhj68mxhzh0000gn/T/ipykernel_83862/1785053538.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  appearnaces_subset['date'] = pd.to_datetime(appearnaces_subset['date'])
/var/folders/xk/dkwzfs8166d33myhj68mxhzh0000gn/T/ipykernel_83862/1785053538.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  appearnaces_subset['season'] = appearnaces_subset['date'].apply(get_football_season)
/var/folders/xk/dkwzfs8166d33myhj68mxhzh0000gn/T/ipykernel_83862/1785053538.py:20: SettingWithCopy

In [5]:
import pandas as pd
from pandas.tseries.offsets import YearEnd

# Convert 'date' to datetime
appearnaces_subset['date'] = pd.to_datetime(appearnaces_subset['date'])

# Step 1: Sort data by player_id and date
appearnaces_subset = appearnaces_subset.sort_values(by=['player_id', 'date']).reset_index(drop=True)

# Step 2: Create a group key for sequential stints
appearnaces_subset['group'] = (
    (appearnaces_subset['team'] != appearnaces_subset['team'].shift()) | 
    (appearnaces_subset['league'] != appearnaces_subset['league'].shift()) | 
    (appearnaces_subset['player_id'] != appearnaces_subset['player_id'].shift())
).cumsum()

# Step 3: Group by player, team, league, and the new group key to preserve sequences
grouped = appearnaces_subset.groupby(['player_id', 'player', 'team', 'league', 'group']).agg(
    start_date=('date', 'min'),
    end_date=('date', 'max'),
).reset_index()

# Drop the 'group' column as it is no longer needed
grouped = grouped.drop(columns='group')

# Step 2: Generate rows for each season
def expand_to_seasons(row):
    seasons = []
    current_date = row['start_date']
    end_date = row['end_date']
    
    while current_date <= end_date:
        season_start = current_date.year if current_date.month >= 7 else current_date.year - 1
        season = f"{season_start}-{season_start + 1}"
        seasons.append({
            'player_id': row['player_id'],
            'player': row['player'],
            'team': row['team'],
            'league': row['league'],
            'season': season,
            #'end_date': row['end_date']
        })
        current_date = pd.Timestamp(f"{season_start + 1}-07-01")
    return seasons

# Step 3: Apply the function to expand seasons
expanded_seasons = grouped.apply(expand_to_seasons, axis=1).explode().reset_index(drop=True)

# Convert list of dictionaries to DataFrame
final_df = pd.DataFrame(expanded_seasons.tolist())

# Step 4: Deduplicate by player_id and season, keeping the most recent end_date
final_df = final_df.sort_values(by=['player_id', 'season'])

In [6]:
# Subset the data to relevant seasons
final_df = final_df[final_df['season'].isin(['2019-2020', '2020-2021', '2021-2022', '2022-2023'])]
final_df

,player_id,player,team,league,season
49,532,Claudio Pizarro,86,Bundesliga,2019-2020
215,2865,Stephan Lichtsteiner,167,Bundesliga,2019-2020
340,3333,James Milner,31,EPL,2019-2020
341,3333,James Milner,31,EPL,2020-2021
342,3333,James Milner,31,EPL,2021-2022
...,...,...,...,...,...
36569,1056655,Rubén Quintanilla,237,LaLiga,2022-2023
36586,1072902,Andrea Dacourt,417,Ligue1,2022-2023
36597,1087022,Serigne Faye,969,Ligue1,2022-2023
36604,1101641,Justin-Noël Kalumba,1420,Ligue1,2022-2023


In [7]:
# Check for a particular player 
final_df[final_df['player'] == 'Cristiano Ronaldo']

,player_id,player,team,league,season
1484,8198,Cristiano Ronaldo,506,SeriaA,2019-2020
1485,8198,Cristiano Ronaldo,506,SeriaA,2020-2021
1486,8198,Cristiano Ronaldo,506,SeriaA,2021-2022
1487,8198,Cristiano Ronaldo,985,EPL,2021-2022
1488,8198,Cristiano Ronaldo,985,EPL,2022-2023


### Map clubs and teams

In [8]:
# Unique mapping file from club to league 
club_to_competition_mapping = clubs[['club_id', 'name', 'domestic_competition_id']].drop_duplicates().sort_values(by='club_id')

# Filter by top5 leagues 
club_to_competition_mapping = club_to_competition_mapping[club_to_competition_mapping['domestic_competition_id'].isin(['ES1', 'IT1', 'L1', 'FR1', 'GB1'])]

# Replace values in a specific column
club_to_competition_mapping['domestic_competition_id'] = club_to_competition_mapping['domestic_competition_id'].replace({
    'ES1': 'LaLiga',
    'IT1': 'SerieA',
    'L1': 'Bundesliga',
    'FR1': 'Ligue1',
    'GB1': 'EPL'
})

club_to_competition_mapping.rename(columns={'name':'team', 'domestic_competition_id':'league'}, inplace=True)

club_to_competition_mapping.head()

,club_id,team,league
310,3,1.FC Köln,Bundesliga
47,4,1.FC Nuremberg,Bundesliga
317,5,Associazione Calcio Milan,SerieA
407,10,Arminia Bielefeld,Bundesliga
329,11,Arsenal Football Club,EPL


In [9]:
club_to_competition_mapping.drop(columns=['league'], inplace=True)

## Merge appearnaces df and get team name 

In [10]:
combined_df = final_df.merge(club_to_competition_mapping, left_on='team', right_on='club_id', how='left')[['season', 'league', 'team_y', 'player', 'player_id']]
combined_df.rename(columns={'team_y':'team'}, inplace=True)

combined_df

,season,league,team,player,player_id
0,2019-2020,Bundesliga,Sportverein Werder Bremen von 1899,Claudio Pizarro,532
1,2019-2020,Bundesliga,FC Augsburg 1907,Stephan Lichtsteiner,2865
2,2019-2020,EPL,Liverpool Football Club,James Milner,3333
3,2020-2021,EPL,Liverpool Football Club,James Milner,3333
4,2021-2022,EPL,Liverpool Football Club,James Milner,3333
...,...,...,...,...,...
11882,2022-2023,LaLiga,Real Club Deportivo Mallorca S.A.D.,Rubén Quintanilla,1056655
11883,2022-2023,Ligue1,Olympique Gymnaste Club Nice Côte d'Azur,Andrea Dacourt,1072902
11884,2022-2023,Ligue1,Montpellier Hérault Sport Club,Serigne Faye,1087022
11885,2022-2023,Ligue1,Angers Sporting Club de l'Ouest,Justin-Noël Kalumba,1101641


## Prepare the valuations data

In [11]:
name_to_id_map = players[['player_id', 'name']].drop_duplicates()

# Merge valuations 
player_valuations = player_valuations.merge(name_to_id_map, how='left', on='player_id')

In [12]:
player_valuations

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id,name
0,405973,2000-01-20,150000,3057,BE1,Fadel Gobitaka
1,342216,2001-07-20,100000,1241,SC1,Julien Serrano
2,3132,2003-12-09,400000,126,TR1,Florin Cernat
3,6893,2003-12-15,900000,984,GB1,Gabriel Tamas
4,10,2004-10-04,7000000,398,IT1,Miroslav Klose
...,...,...,...,...,...,...
487914,1004344,2024-10-18,800000,416,IT1,Ali Dembélé
487915,1077175,2024-10-18,3000000,252,IT1,Honest Ahanor
487916,1083164,2024-10-18,1000000,252,IT1,Lior Kasa
487917,1110645,2024-10-18,500000,398,IT1,Filipe Bordon


In [13]:
# Drop the current club and league 
player_valuations.drop(columns = ['current_club_id', 'player_club_domestic_competition_id'], inplace=True)

# Display data 
player_valuations

,player_id,date,market_value_in_eur,name
0,405973,2000-01-20,150000,Fadel Gobitaka
1,342216,2001-07-20,100000,Julien Serrano
2,3132,2003-12-09,400000,Florin Cernat
3,6893,2003-12-15,900000,Gabriel Tamas
4,10,2004-10-04,7000000,Miroslav Klose
...,...,...,...,...
487914,1004344,2024-10-18,800000,Ali Dembélé
487915,1077175,2024-10-18,3000000,Honest Ahanor
487916,1083164,2024-10-18,1000000,Lior Kasa
487917,1110645,2024-10-18,500000,Filipe Bordon


In [14]:
player_valuations['date'] = pd.to_datetime(player_valuations['date'])

# Step 2: Define season logic
def get_season(date):
    year = date.year
    if date.month >= 7:  # After July belongs to current year-season
        return f"{year}-{year + 1}"
    else:  # Before July belongs to previous year-season
        return f"{year - 1}-{year}"

# Apply the season function
player_valuations['season'] = player_valuations['date'].apply(get_season)

# Step 3: Group by season and get the latest market value
latest_market_value = player_valuations.sort_values(by=['season', 'date']).groupby(['season', 'name']).last().reset_index()

# Step 4: Select required columns
final_df_mv = latest_market_value[['season', 'player_id', 'name', 'market_value_in_eur']]

final_df_mv

,season,player_id,name,market_value_in_eur
0,1999-2000,405973,Fadel Gobitaka,150000
1,2001-2002,342216,Julien Serrano,100000
2,2003-2004,3132,Florin Cernat,400000
3,2003-2004,6893,Gabriel Tamas,900000
4,2004-2005,3580,Aaron Hughes,3500000
...,...,...,...,...
234765,2024-2025,216603,İrfan Can Kahveci,10000000
234766,2024-2025,74641,İsmail Köybaşı,50000
234767,2024-2025,613725,İsmail Yüksek,14000000
234768,2024-2025,811841,İzzet Çelik,75000


In [15]:
final_df_mv = final_df_mv[final_df_mv['season'].isin(['2019-2020', '2020-2021', '2021-2022', '2022-2023'])]
final_df_mv

,season,player_id,name,market_value_in_eur
143046,2019-2020,254249,Aaron Appindangoyé,1200000
143047,2019-2020,578459,Aaron Bastiaans,150000
143048,2019-2020,427605,Aaron Boupendza,200000
143049,2019-2020,360102,Aaron Comrie,250000
143050,2019-2020,434207,Aaron Connolly,4000000
...,...,...,...,...
219426,2022-2023,595201,İsmail Zehir,50000
219427,2022-2023,330806,İsmail Çokçalış,425000
219428,2022-2023,811841,İzzet Çelik,75000
219429,2022-2023,1014342,Şamil Öztürk,75000


In [16]:
final_df_mv[final_df_mv['name'] == 'Cristiano Ronaldo']

,season,player_id,name,market_value_in_eur
146309,2019-2020,8198,Cristiano Ronaldo,60000000
164461,2020-2021,8198,Cristiano Ronaldo,45000000
184103,2021-2022,8198,Cristiano Ronaldo,30000000
203489,2022-2023,8198,Cristiano Ronaldo,15000000


## Join player-club-league data to valuations

In [17]:
combined_df[combined_df['player'] == 'Paul Pogba']

,season,league,team,player,player_id
2934,2019-2020,EPL,Manchester United Football Club,Paul Pogba,122153
2935,2020-2021,EPL,Manchester United Football Club,Paul Pogba,122153
2936,2021-2022,EPL,Manchester United Football Club,Paul Pogba,122153
2937,2022-2023,SeriaA,Juventus Football Club,Paul Pogba,122153


In [18]:
final_df_mv[final_df_mv['name'] == 'Paul Pogba']

,season,player_id,name,market_value_in_eur
156445,2019-2020,122153,Paul Pogba,80000000
175698,2020-2021,122153,Paul Pogba,60000000
195190,2021-2022,122153,Paul Pogba,48000000
214699,2022-2023,122153,Paul Pogba,15000000


In [19]:
the_great_final_df = combined_df.merge(final_df_mv, left_on=['player', 'season'], right_on=['name', 'season'], how='inner')
the_great_final_df = the_great_final_df[['season', 'league', 'team', 'player', 'market_value_in_eur']].drop_duplicates()
the_great_final_df

,season,league,team,player,market_value_in_eur
0,2019-2020,Bundesliga,Sportverein Werder Bremen von 1899,Claudio Pizarro,400000
1,2019-2020,Bundesliga,FC Augsburg 1907,Stephan Lichtsteiner,800000
2,2019-2020,EPL,Liverpool Football Club,James Milner,6500000
3,2020-2021,EPL,Liverpool Football Club,James Milner,3000000
4,2021-2022,EPL,Liverpool Football Club,James Milner,2000000
...,...,...,...,...,...
11773,2022-2023,Ligue1,AC Ajaccio,Moussa Soumano,900000
11774,2022-2023,LaLiga,Real Club Deportivo Mallorca S.A.D.,Rubén Quintanilla,25000
11775,2022-2023,Ligue1,Montpellier Hérault Sport Club,Serigne Faye,300000
11776,2022-2023,Ligue1,Angers Sporting Club de l'Ouest,Justin-Noël Kalumba,250000


## Export the dataset

In [20]:
the_great_final_df.to_csv('output_data/final_valuations_data.csv')

# END